In [1]:
from audioop import avg
import cmath
from fileinput import filename
from gc import enable
import sys
from matplotlib.pyplot import axis
import pandas as pd
from sklearn.model_selection import KFold
import xgboost as xgb
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score, roc_auc_score
import warnings
warnings.filterwarnings("ignore")
from fastai.tabular.all import *


labels = ['SQL Injection', 'Infilteration', 'DoS attacks-SlowHTTPTest','DoS attacks-GoldenEye', 'Bot', 'DoS attacks-Slowloris','Brute Force -Web', 'DDOS attack-LOIC-UDP', 'Benign']

cat_names = ['Dst Port', 'Protocol']
y_names = 'Label'
cont_names = ['Flow Duration', 'Tot Fwd Pkts',
              'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
              'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
              'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
              'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
              'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
              'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
              'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
              'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
              'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
              'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
              'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
              'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
              'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
              'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Fwd Byts/b Avg',
              'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg', 'Bwd Byts/b Avg',
              'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg', 'Subflow Fwd Pkts',
              'Subflow Fwd Byts', 'Subflow Bwd Pkts', 'Subflow Bwd Byts',
              'Init Fwd Win Byts', 'Init Bwd Win Byts', 'Fwd Act Data Pkts',
              'Fwd Seg Size Min', 'Active Mean', 'Active Std', 'Active Max',
              'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min']

df = pd.read_csv('../dataset/CIC-2018/data_balanced.csv')
procs = [Categorify, FillMissing, Normalize]
y_block = CategoryBlock()
splits = RandomSplitter(valid_pct = 0.3 , seed = None)(range_of(df))
to = TabularPandas(df, procs=procs, cat_names=cat_names, cont_names=cont_names,
                   y_names=y_names, y_block=y_block, splits=splits)
X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()


print('Test dataset',"\n",df[y_names].value_counts(), "\n")

dls = to.dataloaders()
roc_auc = RocAuc(average='weighted')
learn = tabular_learner(dls, layers=[200,100], metrics=[accuracy, Precision(average='weighted'), F1Score(average='weighted'), Recall(average='weighted'), roc_auc])
learn.fit(5, 1e-2)
nn_preds = learn.get_preds()[0]

##### DNN CM
start = time.time()
loss, acc, precision, f1, recall, roc = learn.validate()
elapsed = time.time() - start
interp = ClassificationInterpretation.from_learner(learn)
print("DNN ELAPSED: %f" % (elapsed))
print("Confusion Matrix:\n", interp.confusion_matrix())
print('Accuracy: {:.2f}%; precision: {:.2f}%; F1: {:.2f}%; Recall: {:.2f}%; roc-auc: {:.2f}%; elapsed: {:.2f} s'.format(acc*100, precision*100, f1*100, recall*100, roc*100,  elapsed ))

/home/mltest/my_project/my_project_env/lib/python3.8/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


Test dataset 
 DoS attacks-SlowHTTPTest    20000
Infilteration               20000
DoS attacks-Hulk            20000
DoS attacks-GoldenEye       20000
Benign                      20000
Bot                         20000
DDOS attack-HOIC            20000
DoS attacks-Slowloris       20000
DDOS attack-LOIC-UDP        20000
Brute Force -Web              253
Brute Force -XSS              106
SQL Injection                  37
Name: Label, dtype: int64 



epoch,train_loss,valid_loss,accuracy,precision_score,f1_score,recall_score,roc_auc_score,time
0,0.179708,0.172171,0.915980,0.931020,0.914191,0.915980,0.997076,09:21
1,0.078854,0.195056,0.919953,0.929928,0.919064,0.919953,0.996877,12:48
2,0.086600,0.187006,0.931705,0.939729,0.930200,0.931705,0.996850,09:27
3,0.083276,0.216558,0.931077,0.939909,0.930287,0.931077,0.996520,10:29
4,0.079704,0.172006,0.933368,0.942534,0.932226,0.933368,0.996891,08:43


DNN ELAPSED: 9.439271


Confusion Matrix:
 [[4847    6    0    0    1    0    0    0    0    0 1062    0]
 [   0 6052    0    0    0    0    0    0    0    0    0    0]
 [  22    0   27    0    0    0    0    0    0    0   19    0]
 [  19    0    0   14    0    0    0    0    0    0    0    1]
 [   0    0    0    0 6058    0    0    0    0    0    0    0]
 [   0    0    0    0    0 5951    0    0    5    0    0    0]
 [   0    0    0    0    0    0 5897   26    0    3    0    0]
 [   0    0    0    0    0    0   21 6034    0    0    0    0]
 [   0    0    0    0    0 1904    0    0 4116    0    0    0]
 [   2    0    0    0    0    0    1    0    0 6036    0    0]
 [ 505    0    0    0    1    0    0    0    0    0 5476    2]
 [   2    0    2    0    0    0    0    0    0    0    2    4]]
Accuracy: 93.34%; precision: 94.25%; F1: 93.22%; Recall: 93.34%; roc-auc: 99.69%; elapsed: 9.44 s
